In [289]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import string
import numpy as np
import re

### Recherche de dataset:

In [337]:
html=urlopen( "https://fr.wikipedia.org/wiki/Rabat" ).read()
html2=urlopen( "https://fr.wikipedia.org/wiki/Hay_Ryad" ).read()

In [338]:
clean_text=' '.join(BeautifulSoup(html, "html.parser" ).stripped_strings )
clean_text2=' '.join(BeautifulSoup(html2, "html.parser" ).stripped_strings )

### prétraitement de dataset

In [339]:
lower_text=clean_text.lower()
lower_text2=clean_text2.lower()

In [340]:
def remove_numbers(string):
    regex = re.compile(r"(\d|[\u0660\u0661\u0662\u0663\u0664\u0665\u0666\u0667\u0668\u0669])+")
    return re.sub(regex, ' ', string)

In [341]:
def Clean_Text(string):
    return re.sub('[^a-z]', '', string).strip()

In [342]:
def remove_extra_whitespace(string):
    string = re.sub(r'\s+', ' ', string)
    return re.sub(r"\s{2,}", " ", string).strip()

In [389]:
def remove_urls(string):
    regex = re.compile(r"(http|https|ftp)://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
    return re.sub(regex, ' ', string)

In [395]:
out =remove_urls(lower_text)
out2 =remove_urls(lower_text2)

out = lower_text.translate(str.maketrans('', '', string.punctuation))
out2 = lower_text2.translate(str.maketrans('', '', string.punctuation))

In [397]:
out=re.sub('[^a-z-\s]', '', out).strip()  #Rabat
out=re.sub(r"\s{2,}", " ", out).strip()

out2=re.sub('[^a-z-\s]', '', out2).strip()   #Hay riad
out2=re.sub(r"\s{2,}", " ", out2).strip()

out3=re.sub('[^a-z]', '', out).strip()  

### Etape 1 : Modélisation du texte comme succession de lettres (Chaine de Markov d’ordre 1) :

In [398]:
def indicelettre(lettre):
    if((ord(lettre) >96 and ord(lettre)<=122) or lettre==" " ):
        if(lettre==" "):
            return 0
        else:
            asci=ord(lettre)-96
            return asci
    else:
        return -1

In [399]:
def incrementer(text):
    matrice=np.zeros((27,27))
    for i in range(0,len(text)-1):
        if(indicelettre(text[i])==-1):
            i=i+1
        else:
            if(indicelettre(text[i+1])==-1):
                i=i+1
            else:
                matrice[indicelettre(text[i])][indicelettre(text[i+1])]=matrice[indicelettre(text[i])][indicelettre(text[i+1])]+1
    return matrice      

In [400]:
def addition(m):
    compteur=np.zeros(len(m))
    for i in range(0,len(m)):
            compteur[i]=sum(m[i])
    return compteur

In [401]:
def probabilite(out):
    m=incrementer(out)
    comp=addition(m)
    return m/comp

### Etape 2 : Tester la performence de votre modèle :

In [402]:
def fraction_global(text):
    tentative=np.zeros((27,27))
    p=np.zeros((27,27))
    p=probabilite(text)
    score=np.zeros((27,27))
    matrice=np.zeros((27,27))
    for i in range(0,len(text)-1):
        if(indicelettre(text[i])==-1):
            i=i+1
        else:
            if(indicelettre(text[i+1])==-1):
                i=i+1
            else:
                score[indicelettre(text[i])][indicelettre(text[i+1])]=score[indicelettre(text[i])][indicelettre(text[i+1])]+p[indicelettre(text[i])][indicelettre(text[i+1])]
                tentative[indicelettre(text[i])][indicelettre(text[i+1])]=tentative[indicelettre(text[i])][indicelettre(text[i+1])]+1
    return score/tentative

In [413]:
fraction_G=fraction_global(out2)

C:\Users\Aida TALIH\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


### Etape 3 : Modélisation du texte avec un modèle de Markov d’ordre suppérieur :

In [404]:
def indicelettre_partie3(lettre):
    return ord(lettre)-97

In [405]:
def incrementer_partie3(text):
    matrice=np.zeros((26,26,26,26))
    for i in range(0,len(text)-3):
        matrice[indicelettre_partie3(text[i])][indicelettre_partie3(text[i+1])][indicelettre_partie3(text[i+2])][indicelettre_partie3(text[i+3])]+=1
    return matrice  

In [414]:
matrice_transition=incrementer_partie3(out3)

### Etape 4 : Modélisation du texte comme succession de mots:

In [415]:
def indice_mot(text,mot):
    text1=text.split()
    liste=list(set(text1))
    return liste.index(mot)

In [416]:
def incrementer_mot(text):
    text1=text.split()
    liste=set(text1)
    matrice=np.zeros((len(liste),len(liste)))
    for i in range(0,len(text1)-1):
        matrice[indice_mot(text,text1[i])][indice_mot(text,text1[i+1])]+=1
    return matrice 

In [417]:
def matrice_probabilite_mot(out):
    m=incrementer_mot(out)
    comp=addition(m)
    p=np.zeros((len(m),len(m)))
    for i in range(len(m)):
        for j in range(len(m)):
            p[i][j]=m[i][j]/comp[i]
    return p

### Etape 5 : Génération de texte avec un modèle de Markov:

In [418]:
import random
def uniforme():
    return random.random()

In [419]:
def Generateur_text(t):
    p=matrice_probabilite_mot(t)
    text1=t.split()
    liste=list(set(text1))
    m=incrementer_mot(t)
    ligne_proba=random.randint(0,len(p)-1)
    F=np.cumsum(p[ligne_proba])
    unif=uniforme()
    text_final=""
    for k in range(0,len(text1)):
        for i in range(0,len(F)):
            if(unif<=F[i]):
                text_final=text_final+" "+liste[i]
                j=i
                break
        F=np.cumsum(p[j])
        unif=uniforme()
    return text_final

In [420]:
print("Le texte généré:\n")
print(Generateur_text(out2))

Le texte généré:

 des liens espaces de rfrence ou darticles de football de widad tmara qui a t faite le centre de bienfaisance rgie par le centre de widad tmara le quartier principalement rsidentiel et de larrondissement agdalryad notes et holcim maroc rgion rabatsalknitra ville rabat sal vous pouvez partager vos connaissances en cas de rfrencesliste complte portailarchitecture et de qualit traitant du complexe sportif moulayabdallah galeries photos modifier le code rabat plusieurs entreprises marocaines et dimmeubles de complter larticle en pratique quelles sources hay ryad wikipdia lencyclopdie libre cet article manquant de football y sont aussi domicilis tels que la fdration royale marocaine de rabat capitale du complexe sportif moulayabdallah galeries photos modifier le code voir lhistorique plus de noms article discussion contributions crer un article manquant de plusieurs ministres et urbanismearticles lis portailmarocarticles lis portailmonde arabearticles lis portailmarocartic

C:\Users\Aida TALIH\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
